In [46]:
import glob 
import pandas as pd
from os import scandir

import plotly.express as px
import plotly.graph_objects as go

In [18]:
# MAC

# name_files = scandir("names")
# df = pd.DataFrame()
# i = 0
# for file in name_files:
#     year = file.name[3:7]
#     with open(f"names/{file.name}", "r") as handle:
#         lines = handle.read().splitlines()
#     names = [[year] + line.split(",") for line in lines]
#     df = df.join(pd.DataFrame(names, columns=["year", "name", "gender", "populairity"]))
#     i += 1
#     if i == 2:
#         break
# df


In [19]:
# WINDOWS

name_files = scandir("names")
df = pd.DataFrame()
all_data = pd.DataFrame()
for file in glob.glob("./names/*.txt"):
    df = pd.read_csv(file, sep=',', header=None)
    df["year"] = int(file[11:15])
    all_data = pd.concat([all_data, df])
df = all_data
df.columns = ["name", "sex", "count", "year"]
print(df)

          name sex  count  year
0         Mary   F  72173  1922
1      Dorothy   F  37716  1922
2        Helen   F  32508  1922
3     Margaret   F  26865  1922
4         Ruth   F  23630  1922
...        ...  ..    ...   ...
8941     Wyley   M      5  1937
8942      Ynes   M      5  1937
8943   Yoshito   M      5  1937
8944  Zacarias   M      5  1937
8945     Zelma   M      5  1937

[1957046 rows x 4 columns]


In [22]:
filtr_09_18_M = (df.year <= 2018) & (df.year >= 2009) & (df.sex == 'M')
filtr_09_18_F = (df.year <= 2018) & (df.year >= 2009) & (df.sex == 'F')
print(df.loc[filtr_09_18_M].groupby("name").sum().sort_values(by="count", ascending=False))
print(df.loc[filtr_09_18_F].groupby("name").sum().sort_values(by="count", ascending=False))

            count   year
name                    
Noah       180904  20135
Jacob      173195  20135
William    163811  20135
Liam       161558  20135
Ethan      157359  20135
...           ...    ...
Javere          5   2016
Thorbjorn       5   2017
Javez           5   2010
Dailey          5   2016
Sayhan          5   2017

[25604 rows x 2 columns]
                count   year
name                        
Emma           195305  20135
Sophia         183930  20135
Olivia         183016  20135
Isabella       179015  20135
Ava            156852  20135
...               ...    ...
Naiima              5   2018
Aramide             5   2017
Elizabethanne       5   2015
Elizabel            5   2009
Kayleemarie         5   2011

[34967 rows x 2 columns]


In [215]:
"""
Question 2. How many girls and how many boys were born between 1990 and 2000?
"""
children_1990_2000 = df[df.year.between(1990, 2000, inclusive="both")]
boys_1990_2000 = children_1990_2000[children_1990_2000.sex == "M"]
girls_1990_2000 = children_1990_2000[children_1990_2000.sex == "F"]

boys_total = boys_1990_2000["count"].sum()
girls_total = girls_1990_2000["count"].sum()

difference_percentage = (boys_total / girls_total - 1) * 100

print(f"Between years 1990 and 2000 there were born {boys_total} boys and {girls_total} girls.")
print(f"That makes boys being born {difference_percentage:.2f} percent more frequently.")

Between years 1990 and 2000 there were born 21437454 boys and 19816748 girls.
That makes boys being born 8.18 percent more frequently.


In [261]:
"""
Question 3. Can you observe the trends in the number of births over all years? Are these trends different between girls and boys? Provide relevant charts.
"""
births_by_year = df[["sex", "count", "year"]].groupby(["year", "sex"], as_index=False).sum()

boys_by_year = births_by_year[births_by_year.sex == "M"][["year", "count"]].reset_index(drop=True).sort_index()
girls_by_year = births_by_year[births_by_year.sex == "F"][["year", "count"]].reset_index(drop=True).sort_index()

total_by_year = boys_by_year + girls_by_year
total_by_year["year"] //= 2

min_values_dict = {"year":[], "count":[]}
for i in range(len(boys_by_year)):
    min_value = min(boys_by_year.iloc[i]["count"], girls_by_year.iloc[i]["count"])
    min_values_dict["year"].append(boys_by_year.iloc[i].year)
    min_values_dict["count"].append(min_value)
min_values_by_year = pd.DataFrame(min_values_dict)

fig = go.Figure()
fig.add_trace(go.Scatter(x=min_values_by_year.year, y=min_values_by_year["count"], line=dict(color="rgba(0,0,0,0)"), showlegend=False))
fig.add_trace(go.Scatter(x=boys_by_year.year, y=boys_by_year["count"], line=dict(color="#408cff", width=2),
                        name="Boy births", fill="tonexty", fillcolor="rgba(124, 224, 255, 0.5)"))
fig.add_trace(go.Scatter(x=min_values_by_year.year, y=min_values_by_year["count"], line=dict(color="rgba(0,0,0,0)"), showlegend=False))
fig.add_trace(go.Scatter(x=girls_by_year.year, y=girls_by_year["count"], line=dict(color="#ff32ad", width=2),
                        name="Girl births", fill="tonexty", fillcolor="rgba(255, 150, 222, 0.5)"))
fig.add_trace(go.Scatter(x=total_by_year.year, y=total_by_year["count"], line=dict(color="black", width=1), name="Total births"))
fig.show()
print("""
Across the years there can be some trends observed in number of births.
First thing to notice is that there was a relatively low number of new babies born until 1910
and after that year a demographic boom took the birth statistics to whole new levels
as number of new births quadrupled over the next decade.
Then it has stabilized and after a slight decrease there was another boom in the '50 and '60
that have doubled the amounts from 1920.
After both of these demographic booms we can observe decreases in number of births
but these are not as big as the increases and after time stabilize the statistics
on levels higher than before the boom.
The boom from 1960s is a direct effect of the first one as the 1920 babies were
in their reproduction age during that time.

Those trends are the same for men and women. However, we can observe that in early 20 century
there is slightly more women born and around 1937 this trend turns over and since then, there are men born.
""")


Across the years there can be some trends observed in number of births.
First thing to notice is that there was a relatively low number of new babies born until 1910
and after that year a demographic boom took the birth statistics to whole new levels
as number of new births quadrupled over the next decade.
Then it has stabilized and after a slight decrease there was another boom in the '50 and '60
that have doubled the amounts from 1920.
After both of these demographic booms we can observe decreases in number of births
but these are not as big as the increases and after time stabilize the statistics
on levels higher than before the boom.
The boom from 1960s is a direct effect of the first one as the 1920 babies were
in their reproduction age during that time.

Those trends are the same for men and women. However, we can observe that in early 20 century
there is slightly more women born and around 1937 this trend turns over and since then, there are men born.



In [266]:
"""
Question 4. Comment on statistics for your name. Give as meny insights as possible.
Paweł -> Paul
"""
pauls = df.loc[(df.name == "Paul") & (df.sex == "M")].sort_values(by="year").reset_index(drop=True)

fig = px.line(pauls, x="year", y="count", color="name", labels=("Paul"), title="Boys named Paul")
fig.show()

print("""
The increases in the number of boys named Paul aligns pretty well with the two demographic booms
described in the previous question. Although there is one exception which is a peak in 1963 and 1964
that might be correlated with formation of The Beatles music band and the fact that one of the
members' name is Paul McCartney.
After that, the name started to loose on popularity and had a major decline in early 1970s,
then stabilized for a couple of years and started falling down back again after 1980.
The number of Pauls born in 2018 was almost the same as in 1909 despite the fact that
overall number of births was 7 times as high.
""")


The increases in the number of boys named Paul aligns pretty well with the two demographic booms
described in the previous question. Although there is one exception which is a peak in 1963 and 1964
that might be correlated with formation of The Beatles music band and the fact that one of the
members' name is Paul McCartney.
After that, the name started to loose on popularity and had a major decline in early 1970s,
then stabilized for a couple of years and started falling down back again after 1980.
The number of Pauls born in 2018 was almost the same as in 1909 despite the fact that
overall number of births was 7 times as high.

